### This is my SQL query of my project. Since BigQuery doesn't have a way to export the query as a file, i put the query in the jupyter notebook.

### In this notebook, i will explain what i'm doing on the query. Also i will show the query result and some of the visualization from the query. The visualization i made is on Looker Studio

# Data Understanding

Dataset containing five years of customer orders, resulting in thousands of products sold. The records comprise requests from 2017 to 2021. The data was adapted from the JMP Case Study Library. This dataset contain of 2 tables (orders and supplier)

**orders** = detail for each order

|Variable                       |Description |
|:------------------------------|:-----------|
Customer ID 			|Customer unique code
Customer Status 		|Customer priority
Date Order was placed 		|The date when the review was posted
Delivery Date			|The date when orders are delivered
Order ID			|Order unique code
Product ID 		|Code for each product
Quantity Order	|Number of items ordered
Total Retail Price for This Order	|Total price paid by the customer
Cost Price Per Unit		|Product base price / item manufacturing price

**supplier** = product and supplier information

|Variable                       |Description |
|:------------------------------|:-----------|
Product ID 			|Code for each product
Product Line 		|Product division
Product Category 		|Product subset
Product Group		|Product area
Product Name			|Name of the product
Supplier Country		|Supplier's country of origin
Supplier Name	|Name of the supplier
Supplier ID	|Code for each supplier

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = ('/content/drive/MyDrive/Dataset/Wholesale & Retail/orders.csv')
path1 = ('/content/drive/MyDrive/Dataset/Wholesale & Retail/product-supplier.csv')

In [ ]:
ord = pd.read_csv(path)
ps = pd.read_csv(path1)

In [ ]:
ord.head()

,Customer ID,Customer Status,Date Order was placed,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Retail Price for This Order,Cost Price Per Unit
0,579,Silver,01-Jan-17,07-Jan-17,123002578,220101400106,2,92.6,20.70
1,7574,SILVER,01-Jan-17,05-Jan-17,123004074,210201000009,1,21.7,9.95
2,28861,Gold,01-Jan-17,04-Jan-17,123000871,230100500068,1,1.7,0.80
3,43796,Gold,01-Jan-17,06-Jan-17,123002851,220100100633,1,47.9,24.05
4,54673,Gold,01-Jan-17,04-Jan-17,123003607,220200200043,1,36.9,18.30


In [ ]:
ps.head()

,Product ID,Product Line,Product Category,Product Group,Product Name,Supplier Country,Supplier Name,Supplier ID
0,210100100001,Children,Children Outdoors,"Outdoor things, Kids",Boy's and Girl's Ski Pants with Braces,NO,Scandinavian Clothing A/S,50
1,210100100002,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket,ES,Luna sastreria S.A.,4742
2,210100100003,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket Sidney,NO,Scandinavian Clothing A/S,50
3,210100100004,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Set,NO,Scandinavian Clothing A/S,50
4,210100100005,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Suit,NO,Scandinavian Clothing A/S,50


# Information I Want To Find

1. Total order was made from 2017-2021 to see sales progress
2. Customer status to know how many customers in each class and total customers from all sales
3. Product Line sales, to see the sales progress each year
4. Product Line Revenue, can provide information on what product divisions are most profitable
5. Cost vs Retail price, see how profit compares with production costs

# Query

## 1. Total Order


```
SELECT SUM(Quantity_Ordered) AS total_order
FROM `Project_1.wr_orders`
```
Use **SUM** to calculate the total of item ordered using **Quantity_Ordered** column from **orders** table

**Result**

<img src="https://drive.google.com/uc?export=view&id=1N9Mm9mZKFYsyhZDj9Knqt5A9i9v2KpkO" width="185" height="80" alt="Total Order">

## 2. Customer Status


```
SELECT
  DISTINCT(INITCAP(Customer_Status)) AS status,
  COUNT(DISTINCT(Customer_ID)) AS total
FROM `Project_1.wr_orders`
GROUP BY 1
```

- Since the value from **Customer Status** column is not the same in terms of uppercase and lowercase letters, then i use **INITCAP** to capitalize first word for each value. Followed by **DISTINCT** to make sure it's unique value.

```
DISTINCT(INITCAP(Customer_Status)) AS status
```

- Using **DISTINCT** to ensure the number of customers is not duplicated. Then **COUNT** to count the number of unique customers

```
COUNT(DISTINCT(Customer_ID)) AS total
```

**Result**

<img src="https://drive.google.com/uc?export=view&id=1_jZzkKIwwQBQzhOyqpfrYuPMGLTs2sT6" width="400" height="150" alt="Customer Status">

## 3. Product Line Order


```
SELECT
  ps.Product_Line,
  EXTRACT(YEAR FROM o.Date_Order_was_placed) AS Year,
  SUM(o.Quantity_Ordered) Total_Order
FROM `Project_1.wr_orders` o
LEFT JOIN `Project_1.wr_product_supplier` ps
ON o.Product_ID = ps.Product_ID
GROUP BY 1,2
```
- I want to find the sales growth for each product line in 5 years (2017 - 2021). Since the product line is in different table, of course i need to **JOIN** 2 tables.

```
FROM `Project_1.wr_orders` o
LEFT JOIN `Project_1.wr_product_supplier` ps
ON o.Product_ID = ps.Product_ID
```
- The output i want is **product line**, **year**, **total item sold**. So i need to call out 3 of them.
1. Call out **product line** column from product supplier table
```
ps.Product_Line
```
2. **EXTRACT** the yaer from **Date order was placed** column
```
EXTRACT(YEAR FROM o.Date_Order_was_placed) AS Year
```
3. Calculate the total of ordered item
```
SUM(o.Quantity_Ordered) Total_Order
```
**Result**

<img src="https://drive.google.com/uc?export=view&id=1zLI4EUi7Y4bBf_1mIyMzp6pHSgM4-nKn" width="400" height="200" alt="Product Line Sales Query">

**Visualization**

<img src="https://drive.google.com/uc?export=view&id=11ustImQlBMejY7oss0BZj0JlLwyRvjwx" width="400" height="200" alt="Product Line Sales Chart">

As you can see, the total sales are different between each product line. But we can see that all of them drop in 2020. Maybe it cause of COVID-19. **Then why in 2019 the sales still high?** Because of europe and america country start the lockdown in the end of 2020, so they stil did their normal buy activity until the end of 2019.

But because their vaccinations are going so fast, they were able to lift their lockdown policy earlier. That's why in 2021, the number sales was going up again.

## 4. Product Line Revenue

```
SELECT
  ps.Product_Line,
  EXTRACT(YEAR FROM o.Date_Order_was_placed) AS Year,
  ROUND(SUM(o.Total_Retail_Price_for_This_Order
  - (o.Cost_Price_Per_Unit * o.Quantity_Ordered))) AS profit
FROM `Project_1.wr_orders` o
LEFT JOIN `Project_1.wr_product_supplier` ps
ON o.Product_ID = ps.Product_ID
GROUP BY 1,2
```

-  Same as number 3, i need to **JOIN** 2 tables to get the product line column
```
FROM `Project_1.wr_orders` o
LEFT JOIN `Project_1.wr_product_supplier` ps
ON o.Product_ID = ps.Product_ID
```
- The output i want is **product line**, **year**, **total profit**. So i need to call out 3 of them.
1. Call out **product line** from product supplier table
```
ps.Product_Line
```
2. **EXTRACT** the yaer from **Date order was placed** column
```
EXTRACT(YEAR FROM o.Date_Order_was_placed) AS Year
```
3. Calculate the profit each order, use **ROUND** to get compact number   
 profit = retail price / (quantity * cost price)
```
ROUND(SUM(o.Total_Retail_Price_for_This_Order
  - (o.Cost_Price_Per_Unit * o.Quantity_Ordered))) AS profit
```
**Result**

<img src="https://drive.google.com/uc?export=view&id=1VychhvDbcrl-b9SK-SunIUmCnA8VWS6l" width="400" height="200" alt="Product Line Revenue Query">

**Visualization**

<img src="https://drive.google.com/uc?export=view&id=1P31W-1Q4DXFjG_UonkgOsWU9CNBE1IgY" width="400" height="200" alt="Product Line Revenue Chart">

Same as number 3, the revenue went down in 2020 but went up again in 2021.

## 5. Profit Vs Cost

```
SELECT
  ps.Product_Line,
  ROUND(SUM(o.Cost_Price_Per_Unit * o.Quantity_Ordered)) AS Cost,
  ROUND(SUM(o.Total_Retail_Price_for_This_Order)) AS Retail
FROM `Project_1.wr_orders` o
LEFT JOIN `Project_1.wr_product_supplier` ps
ON o.Product_ID = ps.Product_ID
GROUP BY 1
```

- **JOIN** 2 tables to get product line column
```
ps.Product_Line
```

- The cost price provided is for 1 item, so i need to multiply it with quantity of order. Calculate the total, then **ROUND** to get compact number
```
ROUND(SUM(o.Cost_Price_Per_Unit * o.Quantity_Ordered)) AS Cost
```

- And i need to call out the total retail price, so i can compare it with the total cost price. **ROUND** again to get compact number
```
ROUND(SUM(o.Total_Retail_Price_for_This_Order)) AS Retail
```

**Result**

<img src="https://drive.google.com/uc?export=view&id=13UTF4PCkNa6HGU48-_Fl7fPM_XbpzYdi" width="350" height="150" alt="Profit vs Cost Query">

**Visualization**

<img src="https://drive.google.com/uc?export=view&id=1d5Jl9VmnLBZoa_sY7JW_0BrsxX5R3XZ5" width="500" height="200" alt="Profit vs Cost Chart">

# Insight

1. If you look at the total sales and profit charts, both tend to have similar graphs. Where sales and profits rise every year, but there was a decline in 2020 due to COVID-19. But everything went back up in 2021, and is expected to continue to rise every year.

2. Each product division has the same percentage of profit, which is about 70%.